In [4]:
# drop_from_originals.py
from pathlib import Path
import pandas as pd

# 1) 드랍 대상
DROP_ITEMS = {"애느타리(일반)", "건표고버섯", "꽃느타리버섯", "생표고(수입)"}
CANDIDATE_COLS = ["품목명", "세부 품목", "세부품목", "품목"]

# 2) 입력 파일 수집: 현재 폴더와 ./01 수민 모두 탐색
candidates = [Path("."), Path("./01 수민")]
files = []
for root in candidates:
    if root.exists():
        for p in sorted(root.glob("*.csv")):
            if not p.name.endswith("_cleaned.csv"):  # 결과물 재처리 방지
                files.append(p.resolve())

# 중복 제거(동일 경로가 두 번 잡히는 경우 방지)
files = list(dict.fromkeys(files))

print(f"[INFO] CWD: {Path.cwd().resolve()}")
print(f"[INFO] 찾은 CSV: {len(files)}개")
if not files:
    raise SystemExit("[중지] 처리할 CSV를 찾지 못했습니다. 실행 위치/경로를 확인하세요.")

total_in = total_out = 0
for fp in files:
    # 3) 읽기 (인코딩 자동 대체)
    try:
        df = pd.read_csv(fp, encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(fp, encoding="cp949")

    # 칼럼 공백 제거
    df.columns = df.columns.str.strip()

    # 4) 세부품목 컬럼 탐지
    target_col = next((c for c in CANDIDATE_COLS if c in df.columns), None)
    if target_col is None:
        print(f"[SKIP] {fp.name}: 세부품목 컬럼을 찾지 못함 -> 보유칼럼 예시: {list(df.columns)[:8]}")
        continue

    before = len(df)
    df2 = df[~df[target_col].astype(str).isin(DROP_ITEMS)].copy()
    after = len(df2)

    # 5) 저장: 원본 파일이 있는 폴더 아래 cleaned/
    out_dir = fp.parent / "cleaned"
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"{fp.stem}_cleaned.csv"
    df2.to_csv(out_path, index=False, encoding="utf-8-sig")

    total_in += before
    total_out += after
    dropped = before - after
    print(f"[OK] {fp.name}: {before} → {after} (드랍 {dropped}행) | 사용컬럼: {target_col} | 저장: {out_path}")

print(f"\n[요약] {len(files)}개 파일 처리 완료. 총 {total_in}행 → {total_out}행")
print("[완료] 각 폴더의 cleaned/ 안을 확인하세요.")


[INFO] CWD: /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/01 수민
[INFO] 찾은 CSV: 5개
[OK] 느타리버섯.csv: 14616 → 10962 (드랍 3654행) | 사용컬럼: 품목명 | 저장: /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/01 수민/cleaned/느타리버섯_cleaned.csv
[OK] 새송이버섯.csv: 3654 → 3654 (드랍 0행) | 사용컬럼: 품목명 | 저장: /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/01 수민/cleaned/새송이버섯_cleaned.csv
[OK] 양송이버섯.csv: 3654 → 3654 (드랍 0행) | 사용컬럼: 품목명 | 저장: /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/01 수민/cleaned/양송이버섯_cleaned.csv
[OK] 팽이버섯.csv: 3654 → 3654 (드랍 0행) | 사용컬럼: 품목명 | 저장: /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/01 수민/cleaned/팽이버섯_cleaned.csv
[OK] 표고버섯.csv: 10962 → 7308 (드랍 3654행) | 사용컬럼: 품목명 | 저장: /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/01 수민/cleaned/표고버섯_cleaned.csv

[요약] 5개 파일 처리 완료. 총 36540행 → 29232행
[완료] 각 폴더의 cleaned/ 안을 확인하세요.
